[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_21/iris_neural_network.ipynb)

# Neuronale Netze mit dem Iris-Datensatz 🌸

**Ziel:** Klassifikation von Iris-Blüten mit einem neuronalen Netz

In diesem Notebook lernen wir:
- Wie man den klassischen Iris-Datensatz für neuronale Netze vorbereitet
- Aufbau und Training eines Multi-Layer Perceptrons (MLP)
- Evaluation und Interpretation der Ergebnisse
- Praktische Tipps für neuronale Netze bei kleinen Datensätzen

**Der Iris-Datensatz:** 150 Iris-Blüten, 4 Features (Kelch-/Blütenblattlänge und -breite), 3 Klassen (Setosa, Versicolor, Virginica)

## 1. Import Required Libraries

Wir benötigen verschiedene Bibliotheken für Datenverarbeitung, neuronale Netze und Visualisierung.

In [ ]:
# Grundlegende Bibliotheken
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn für Daten und Metriken
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Für bessere Plots
plt.style.use('default')
sns.set_palette("husl")

# Reproduzierbarkeit
np.random.seed(43)

print("✅ Alle Bibliotheken erfolgreich importiert!")

## 2. Load and Explore the Iris Dataset

Der Iris-Datensatz ist ein Klassiker des Machine Learning. Lass uns ihn laden und verstehen.

In [ ]:
# Iris-Datensatz laden
iris = load_iris()

# In DataFrame konvertieren für bessere Handhabung
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = iris.target
df['species_name'] = df['species'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

print("📊 Iris-Datensatz Überblick:")
print(f"Shape: {df.shape}")
print(f"Features: {list(iris.feature_names)}")
print(f"Klassen: {list(iris.target_names)}")
print("\nErste 5 Zeilen:")
print(df.head())

In [ ]:
# Datenverteilung visualisieren
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Iris Dataset - Feature Distributions', fontsize=16, fontweight='bold')

# Pairplot-ähnliche Visualisierung der wichtigsten Features
features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
colors = ['red', 'green', 'blue']

for i, feature in enumerate(features):
    ax = axes[i//2, i%2]
    for j, species in enumerate(['setosa', 'versicolor', 'virginica']):
        data = df[df['species_name'] == species][feature]
        ax.hist(data, alpha=0.7, label=species, color=colors[j], bins=15)
    
    ax.set_title(f'{feature}')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistische Summary
print("\n📈 Statistische Zusammenfassung:")
print(df.groupby('species_name')[features].mean().round(2))

## 3. Data Preprocessing and Splitting

Neuronale Netze funktionieren am besten mit standardisierten Daten. Wir skalieren die Features und teilen die Daten auf.

In [ ]:
# Features (X) und Zielwerte (y) trennen
X = iris.data  # 4 Features: sepal length/width, petal length/width
y = iris.target  # 3 Klassen: 0=setosa, 1=versicolor, 2=virginica

print("🔍 Datenform vor Preprocessing:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Klassen: {np.unique(y)}")

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\n📊 Datenaufteilung:")
print(f"Training: {X_train.shape[0]} Samples")
print(f"Test: {X_test.shape[0]} Samples")

# Feature Scaling - WICHTIG für neuronale Netze!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n⚖️ Feature-Skalierung:")
print("Vor Skalierung (Training):")
print(f"Mean: {X_train.mean(axis=0).round(2)}")
print(f"Std:  {X_train.std(axis=0).round(2)}")
print("\nNach Skalierung (Training):")
print(f"Mean: {X_train_scaled.mean(axis=0).round(2)}")
print(f"Std:  {X_train_scaled.std(axis=0).round(2)}")

## 4. Build Neural Network Model

Jetzt erstellen wir unser neuronales Netz! Wir verwenden ein Multi-Layer Perceptron (MLP) mit einer versteckten Schicht.

In [ ]:
# Neuronales Netz definieren
mlp = MLPClassifier(
    hidden_layer_sizes=(10, 5),    # Zwei versteckte Schichten: 10 und 5 Neuronen
    activation='relu',             # ReLU-Aktivierungsfunktion
    solver='adam',                 # Adam-Optimizer (modern und effizient)
    alpha=0.001,                   # L2-Regularisierung
    batch_size='auto',             # Automatische Batch-Größe
    learning_rate='constant',      # Konstante Lernrate
    learning_rate_init=0.001,      # Anfängliche Lernrate
    max_iter=1000,                 # Maximale Iterationen
    shuffle=True,                  # Daten in jeder Epoche mischen
    random_state=42,               # Reproduzierbarkeit
    verbose=True                   # Training-Fortschritt anzeigen
)

print("🧠 Neuronales Netz Architektur:")
print("Input Layer:    4 Neuronen (Features)")
print("Hidden Layer 1: 10 Neuronen (ReLU)")
print("Hidden Layer 2: 5 Neuronen (ReLU)")
print("Output Layer:   3 Neuronen (Softmax)")
print("\n🔧 Hyperparameter:")
print(f"Aktivierung: {mlp.activation}")
print(f"Optimizer: {mlp.solver}")
print(f"Lernrate: {mlp.learning_rate_init}")
print(f"Regularisierung: {mlp.alpha}")
print(f"Max. Iterationen: {mlp.max_iter}")

## 5. Train the Neural Network

Zeit zum Training! Das neuronale Netz lernt die Muster in den Iris-Daten.

In [ ]:
# Training starten
print("🚀 Training gestartet...")
import time
start_time = time.time()

# Model trainieren
mlp.fit(X_train_scaled, y_train)

training_time = time.time() - start_time
print(f"\n✅ Training abgeschlossen in {training_time:.2f} Sekunden")
print(f"📊 Anzahl Iterationen: {mlp.n_iter_}")
print(f"🎯 Training konvergiert: {'Ja' if mlp.n_iter_ < mlp.max_iter else 'Nein'}")

# Training-Loss visualisieren
plt.figure(figsize=(10, 6))
plt.plot(mlp.loss_curve_, 'b-', linewidth=2)
plt.title('Training Loss Curve', fontsize=14, fontweight='bold')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.show()

print(f"📉 Final Loss: {mlp.loss_curve_[-1]:.6f}")

## 5.5. Cross-Validation für Generalisierungsfehler

Bevor wir das finale Test Set evaluieren, führen wir eine 5-fach Cross-Validation durch, um den Generalisierungsfehler zu schätzen.

In [ ]:
# Cross-Validation für robuste Performance-Schätzung
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.neural_network import MLPClassifier

print("🔄 Cross-Validation: 3-fach vs. 5-fach vs. 8-fach vs. 10-fach Vergleich")
print("💡 Bei kleinen Datensätzen entstehen verschiedene Probleme je nach Fold-Anzahl")

# Neues MLPClassifier-Objekt für CV (gleiche Parameter)
mlp_cv = MLPClassifier(
    hidden_layer_sizes=(10, 5),
    activation='relu',
    solver='adam',
    learning_rate_init=0.01,
    alpha=0.0001,
    max_iter=1000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10
)

print(f"\n📊 Datensatz-Analyse - Warum Fold-Größe wichtig ist:")
print(f"Gesamte Trainingsdaten: {len(X_train)} Samples")
print(f"Bei 3-fach CV: ~{len(X_train)//3} Samples pro Fold ({len(X_train)//3/len(X_train)*100:.1f}% als Test) - Große Folds")
print(f"Bei 5-fach CV: ~{len(X_train)//5} Samples pro Fold ({len(X_train)//5/len(X_train)*100:.1f}% als Test) - Moderate Folds")
print(f"Bei 8-fach CV: ~{len(X_train)//8} Samples pro Fold ({len(X_train)//8/len(X_train)*100:.1f}% als Test) - Kleine Folds")
print(f"Bei 10-fach CV: ~{len(X_train)//10} Samples pro Fold ({len(X_train)//10/len(X_train)*100:.1f}% als Test) - Sehr kleine Folds")

# 3-fach Cross-Validation
print("\n🔄 3-fach Cross-Validation...")
skf_3 = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
cv_scores_3 = cross_val_score(mlp_cv, X_train_scaled, y_train, cv=skf_3, scoring='accuracy', n_jobs=-1)

# 5-fach Cross-Validation
print("🔄 5-fach Cross-Validation...")
skf_5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_5 = cross_val_score(mlp_cv, X_train_scaled, y_train, cv=skf_5, scoring='accuracy', n_jobs=-1)

# 8-fach Cross-Validation
print("🔄 8-fach Cross-Validation...")
skf_8 = StratifiedKFold(n_splits=8, shuffle=True, random_state=42)
cv_scores_8 = cross_val_score(mlp_cv, X_train_scaled, y_train, cv=skf_8, scoring='accuracy', n_jobs=-1)

# 10-fach Cross-Validation  
print("🔄 10-fach Cross-Validation...")
skf_10 = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores_10 = cross_val_score(mlp_cv, X_train_scaled, y_train, cv=skf_10, scoring='accuracy', n_jobs=-1)

# Statistiken berechnen
cv_mean_3, cv_std_3 = cv_scores_3.mean(), cv_scores_3.std()
cv_mean_5, cv_std_5 = cv_scores_5.mean(), cv_scores_5.std()
cv_mean_8, cv_std_8 = cv_scores_8.mean(), cv_scores_8.std()
cv_mean_10, cv_std_10 = cv_scores_10.mean(), cv_scores_10.std()

print("📊 Cross-Validation Ergebnisse Vergleich:")
print("\n🎯 3-fach CV:")
print(f"Fold Accuracies: {cv_scores_3}")
print(f"Mittelwert: {cv_mean_3:.4f} ± {cv_std_3:.4f}")
print(f"Bereich: {cv_scores_3.min():.4f} - {cv_scores_3.max():.4f}")
print(f"Variationskoeffizient: {(cv_std_3/cv_mean_3)*100:.2f}%")

print("\n🎯 5-fach CV:")
print(f"Fold Accuracies: {cv_scores_5}")
print(f"Mittelwert: {cv_mean_5:.4f} ± {cv_std_5:.4f}")
print(f"Bereich: {cv_scores_5.min():.4f} - {cv_scores_5.max():.4f}")
print(f"Variationskoeffizient: {(cv_std_5/cv_mean_5)*100:.2f}%")

print("\n🎯 8-fach CV:")
print(f"Fold Accuracies: {cv_scores_8}")
print(f"Mittelwert: {cv_mean_8:.4f} ± {cv_std_8:.4f}")
print(f"Bereich: {cv_scores_8.min():.4f} - {cv_scores_8.max():.4f}")
print(f"Variationskoeffizient: {(cv_std_8/cv_mean_8)*100:.2f}%")

print("\n🎯 10-fach CV:")
print(f"Fold Accuracies: {cv_scores_10}")
print(f"Mittelwert: {cv_mean_10:.4f} ± {cv_std_10:.4f}")
print(f"Bereich: {cv_scores_10.min():.4f} - {cv_scores_10.max():.4f}")
print(f"Variationskoeffizient: {(cv_std_10/cv_mean_10)*100:.2f}%")

# Stabilität korrekt analysieren - welche Methode hat die NIEDRIGSTE Varianz?
stds = [cv_std_3, cv_std_5, cv_std_8, cv_std_10]
methods = ['3-fach', '5-fach', '8-fach', '10-fach']
most_stable_idx = np.argmin(stds)
least_stable_idx = np.argmax(stds)

print(f"\n📈 Stabilität-Ranking (niedrigste → höchste Varianz):")
sorted_indices = np.argsort(stds)
for i, idx in enumerate(sorted_indices):
    status = ""
    if i == 0:
        status = " ⭐ STABILSTE"
    elif i == len(sorted_indices) - 1:
        status = " ❌ INSTABILSTE" 
    print(f"{i+1}. {methods[idx]}: {stds[idx]:.4f}{status}")

print(f"\n🎯 Beobachtung:")
print(f"In diesem Datensatz zeigt {methods[most_stable_idx]} CV die niedrigste Varianz.")
print(f"{methods[least_stable_idx]} CV zeigt die höchste Varianz - die Folds sind {'sehr groß' if least_stable_idx == 0 else 'sehr klein'}.")

# Cross-Validation visualisieren - jetzt mit 4 Methoden!
plt.figure(figsize=(20, 15))

# Subplot 1: 3-fach CV Fold-Ergebnisse
plt.subplot(3, 3, 1)
fold_numbers_3 = range(1, 4)
bars_3 = plt.bar(fold_numbers_3, cv_scores_3, color='lightcoral', alpha=0.7, edgecolor='darkred')
plt.axhline(y=cv_mean_3, color='red', linestyle='--', linewidth=2, label=f'Mean: {cv_mean_3:.4f}')
plt.axhline(y=cv_mean_3 + cv_std_3, color='orange', linestyle=':', alpha=0.7, label=f'±1 Std: {cv_std_3:.4f}')
plt.axhline(y=cv_mean_3 - cv_std_3, color='orange', linestyle=':', alpha=0.7)
plt.title('3-fach CV: Fold Accuracies', fontweight='bold')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
for bar, score in zip(bars_3, cv_scores_3):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{score:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=10)

# Subplot 2: 5-fach CV Fold-Ergebnisse
plt.subplot(3, 3, 2)
fold_numbers_5 = range(1, 6)
bars_5 = plt.bar(fold_numbers_5, cv_scores_5, color='skyblue', alpha=0.7, edgecolor='navy')
plt.axhline(y=cv_mean_5, color='red', linestyle='--', linewidth=2, label=f'Mean: {cv_mean_5:.4f}')
plt.axhline(y=cv_mean_5 + cv_std_5, color='orange', linestyle=':', alpha=0.7, label=f'±1 Std: {cv_std_5:.4f}')
plt.axhline(y=cv_mean_5 - cv_std_5, color='orange', linestyle=':', alpha=0.7)
plt.title('5-fach CV: Fold Accuracies', fontweight='bold')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
for bar, score in zip(bars_5, cv_scores_5):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{score:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# Subplot 3: 8-fach CV Fold-Ergebnisse
plt.subplot(3, 3, 3)
fold_numbers_8 = range(1, 9)
bars_8 = plt.bar(fold_numbers_8, cv_scores_8, color='lightseagreen', alpha=0.7, edgecolor='darkgreen')
plt.axhline(y=cv_mean_8, color='red', linestyle='--', linewidth=2, label=f'Mean: {cv_mean_8:.4f}')
plt.axhline(y=cv_mean_8 + cv_std_8, color='orange', linestyle=':', alpha=0.7, label=f'±1 Std: {cv_std_8:.4f}')
plt.axhline(y=cv_mean_8 - cv_std_8, color='orange', linestyle=':', alpha=0.7)
plt.title('8-fach CV: Fold Accuracies', fontweight='bold')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
for bar, score in zip(bars_8, cv_scores_8):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{score:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=8)

# Subplot 4: 10-fach CV Fold-Ergebnisse
plt.subplot(3, 3, 4)
fold_numbers_10 = range(1, 11)
bars_10 = plt.bar(fold_numbers_10, cv_scores_10, color='plum', alpha=0.7, edgecolor='purple')
plt.axhline(y=cv_mean_10, color='red', linestyle='--', linewidth=2, label=f'Mean: {cv_mean_10:.4f}')
plt.axhline(y=cv_mean_10 + cv_std_10, color='orange', linestyle=':', alpha=0.7, label=f'±1 Std: {cv_std_10:.4f}')
plt.axhline(y=cv_mean_10 - cv_std_10, color='orange', linestyle=':', alpha=0.7)
plt.title('10-fach CV: Fold Accuracies', fontweight='bold')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
for bar, score in zip(bars_10, cv_scores_10):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{score:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=7)

# Subplot 5: Boxplot Vergleich aller vier
plt.subplot(3, 3, 5)
box_plot = plt.boxplot([cv_scores_3, cv_scores_5, cv_scores_8, cv_scores_10], 
                      labels=['3-fach', '5-fach', '8-fach', '10-fach'], patch_artist=True)
colors = ['lightcoral', 'skyblue', 'lightseagreen', 'plum']
for box, color in zip(box_plot['boxes'], colors):
    box.set_facecolor(color)
    box.set_alpha(0.7)
plt.title('CV Score Distributions Vergleich', fontweight='bold')
plt.ylabel('Accuracy')
plt.grid(True, alpha=0.3)

# Subplot 6: Stabilität Analyse (Mittelwert ± Std)
plt.subplot(3, 3, 6)
methods_plot = ['3-fach', '5-fach', '8-fach', '10-fach']
means_plot = [cv_mean_3, cv_mean_5, cv_mean_8, cv_mean_10]
stds_plot = [cv_std_3, cv_std_5, cv_std_8, cv_std_10]
x_pos = range(len(methods_plot))

bars = plt.bar(x_pos, means_plot, yerr=stds_plot, capsize=5, 
               color=colors, alpha=0.7, 
               edgecolor=['darkred', 'navy', 'darkgreen', 'purple'])
plt.title('Mittelwert ± Standardabweichung', fontweight='bold')
plt.xlabel('Methode')
plt.ylabel('Accuracy')
plt.xticks(x_pos, methods_plot, rotation=45)
plt.grid(True, alpha=0.3)

# Werte anzeigen
for i, (bar, mean, std) in enumerate(zip(bars, means_plot, stds_plot)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.01,
             f'{mean:.4f}\n±{std:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=8)

# Subplot 7: Standardabweichung Trends - korrigiert!
plt.subplot(3, 3, 7)
folds_all = [3, 5, 8, 10]
std_values_all = [cv_std_3, cv_std_5, cv_std_8, cv_std_10]
plt.plot(folds_all, std_values_all, 'ro-', linewidth=2, markersize=8, label='Standardabweichung')
plt.title('CV-Stabilität vs. Anzahl Folds (Korrigiert!)', fontweight='bold')
plt.xlabel('Anzahl Folds')
plt.ylabel('Standardabweichung')
plt.grid(True, alpha=0.3)
plt.legend()

# Trend-Annotation
for fold, std_val in zip(folds_all, std_values_all):
    plt.annotate(f'{std_val:.4f}', (fold, std_val), 
                textcoords="offset points", xytext=(0,10), ha='center', fontweight='bold')

# Optimal-Zone markieren
min_std_idx = np.argmin(std_values_all)
optimal_folds = folds_all[min_std_idx]
plt.axvline(x=optimal_folds, color='green', linestyle='--', alpha=0.7, 
           label=f'Optimal: {optimal_folds}-fach')
plt.legend()

# Subplot 8: Variationskoeffizient Vergleich
plt.subplot(3, 3, 8)
cv_coeffs = [(std/mean)*100 for std, mean in zip(stds_plot, means_plot)]
bars_cv = plt.bar(methods_plot, cv_coeffs, color=colors, alpha=0.7)
plt.title('Variationskoeffizient (%)', fontweight='bold')
plt.xlabel('Methode')
plt.ylabel('Variationskoeffizient (%)')
plt.grid(True, alpha=0.3)
for bar, cv_coeff in zip(bars_cv, cv_coeffs):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{cv_coeff:.2f}%', ha='center', va='bottom', fontweight='bold')

# Subplot 9: Stabilität-Ranking
plt.subplot(3, 3, 9)
sorted_indices = np.argsort(stds_plot)
ranking_methods = [methods_plot[i] for i in sorted_indices]
ranking_stds = [stds_plot[i] for i in sorted_indices]
ranking_colors = [colors[i] for i in sorted_indices]

bars_rank = plt.bar(range(len(ranking_methods)), ranking_stds, 
                   color=ranking_colors, alpha=0.7)
plt.title('Stabilität-Ranking (Niedrigste → Höchste Varianz)', fontweight='bold')
plt.xlabel('Ranking')
plt.ylabel('Standardabweichung')
plt.xticks(range(len(ranking_methods)), ranking_methods)
plt.grid(True, alpha=0.3)

# Ranking Labels
for i, (bar, std_val, method) in enumerate(zip(bars_rank, ranking_stds, ranking_methods)):
    label = "🏆 BESTE" if i == 0 else "❌ SCHLECHTESTE" if i == len(ranking_methods)-1 else ""
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{std_val:.4f}\n{label}', ha='center', va='bottom', fontweight='bold', fontsize=8)

plt.tight_layout()
plt.show()

# Detaillierte Analyse warum CV bei kleinen Datensätzen problematisch sein kann
print("\n🔬 Analyse: Warum ist CV bei kleinen Datensätzen instabil?")
print("=" * 60)

train_samples = len(X_train)
samples_per_fold_5 = train_samples // 5
samples_per_fold_10 = train_samples // 10

print(f"\n📊 Datensatz-Größe Problematik:")
print(f"Training Samples: {train_samples}")

samples_per_fold_3 = train_samples // 3
samples_per_fold_8 = train_samples // 8
print(f"Test Samples pro Fold (3-fach): ~{samples_per_fold_3} ({samples_per_fold_3/train_samples*100:.1f}%) - Sehr große Test-Folds")
print(f"Test Samples pro Fold (5-fach): ~{samples_per_fold_5} ({samples_per_fold_5/train_samples*100:.1f}%) - Moderate Test-Folds")
print(f"Test Samples pro Fold (8-fach): ~{samples_per_fold_8} ({samples_per_fold_8/train_samples*100:.1f}%) - Kleine Test-Folds")
print(f"Test Samples pro Fold (10-fach): ~{samples_per_fold_10} ({samples_per_fold_10/train_samples*100:.1f}%) - Sehr kleine Test-Folds")

# Korrekte Variabilität-Analyse basierend auf den tatsächlichen Ergebnissen
stds_analysis = [cv_std_3, cv_std_5, cv_std_8, cv_std_10]
methods_analysis = ['3-fach', '5-fach', '8-fach', '10-fach']
most_stable_idx = np.argmin(stds_analysis)
least_stable_idx = np.argmax(stds_analysis)

print(f"\n📈 KORRIGIERTE Variabilität-Analyse:")
print(f"3-fach CV Standardabweichung:  {cv_std_3:.4f}")
print(f"5-fach CV Standardabweichung:  {cv_std_5:.4f}")
print(f"8-fach CV Standardabweichung:  {cv_std_8:.4f}")
print(f"10-fach CV Standardabweichung: {cv_std_10:.4f}")

print(f"\n🎯 Tatsächliches Ranking (stabilste → instabilste):")
sorted_indices = np.argsort(stds_analysis)
for i, idx in enumerate(sorted_indices):
    status = " 🏆 STABILSTE" if i == 0 else " ❌ INSTABILSTE" if i == len(sorted_indices)-1 else ""
    print(f"{i+1}. {methods_analysis[idx]} CV: {stds_analysis[idx]:.4f}{status}")

print(f"\n💡 Mögliche Erklärungen:")
if most_stable_idx == 1:  # 5-fach
    print("• 5-fach CV: Balance zwischen Fold-Größe (~21 Samples) und Wiederholungen (5x)")
elif most_stable_idx == 2:  # 8-fach
    print("• 8-fach CV: Kompromiss zwischen Fold-Größe (~13 Samples) und Wiederholungen (8x)")

if least_stable_idx == 3:  # 10-fach
    print("• 10-fach CV: Test-Folds sehr klein (~10 Samples) → möglicherweise zu wenig für stabile Schätzung")
elif least_stable_idx == 0:  # 3-fach
    print("• 3-fach CV: Wenige Wiederholungen (nur 3x) → möglicherweise unzuverlässige Durchschnittsbildung")

# Konfidenzintervalle vergleichen
ci_width_3 = 1.96 * cv_std_3
ci_width_5 = 1.96 * cv_std_5
ci_width_8 = 1.96 * cv_std_8
ci_width_10 = 1.96 * cv_std_10

print(f"\n🎯 95% Konfidenzintervalle:")
print(f"3-fach CV:  [{cv_mean_3-ci_width_3:.4f}, {cv_mean_3+ci_width_3:.4f}] (Breite: {2*ci_width_3:.4f})")
print(f"5-fach CV:  [{cv_mean_5-ci_width_5:.4f}, {cv_mean_5+ci_width_5:.4f}] (Breite: {2*ci_width_5:.4f})")
print(f"8-fach CV:  [{cv_mean_8-ci_width_8:.4f}, {cv_mean_8+ci_width_8:.4f}] (Breite: {2*ci_width_8:.4f})")
print(f"10-fach CV: [{cv_mean_10-ci_width_10:.4f}, {cv_mean_10+ci_width_10:.4f}] (Breite: {2*ci_width_10:.4f})")

# Beste Methode identifizieren
best_method = methods_analysis[most_stable_idx]
best_ci_width = [ci_width_3, ci_width_5, ci_width_8, ci_width_10][most_stable_idx]
print(f"\n📏 {best_method} CV hat das schmalste Konfidenzintervall: {2*best_ci_width:.4f}")

# Probleme bei Cross-Validation mit kleinen Datensätzen
print(f"\n💡 Probleme bei Cross-Validation mit kleinen Datensätzen:")
print("📊 Grundproblem: Bei nur 105 Trainingssamples entstehen verschiedene Herausforderungen:")

print(f"\n🔍 Analyse der Fold-Größen:")
print("• 3-fach CV: ~35 Test-Samples, ~70 Training-Samples pro Fold")
print("  → Wenige Wiederholungen, aber größere Folds")
print("• 5-fach CV: ~21 Test-Samples, ~84 Training-Samples pro Fold") 
print("  → Moderate Fold-Größen")
print("• 8-fach CV: ~13 Test-Samples, ~92 Training-Samples pro Fold")
print("  → Kleinere Test-Folds")
print("• 10-fach CV: ~10 Test-Samples, ~95 Training-Samples pro Fold")
print("  → Sehr kleine Test-Folds")

print(f"\n📈 Trade-offs bei kleinen Datensätzen:")
print("📌 Wenige Folds (z.B. 3): Größere Test-Sets pro Fold, aber weniger Wiederholungen")
print("📌 Viele Folds (z.B. 10): Mehr Wiederholungen, aber sehr kleine Test-Sets pro Fold")
print("📌 Kleine Test-Sets: Höhere Varianz der Schätzungen zwischen den Folds")
print("📌 Wenige Wiederholungen: Weniger robuste Durchschnittsbildung")

optimal_method = methods_analysis[most_stable_idx]
print(f"\n📊 Beobachtung für diesen Datensatz:")
print(f"Die Ergebnisse zeigen, dass {optimal_method} CV die geringste Varianz aufweist.")
print("Dies kann bei anderen kleinen Datensätzen anders ausfallen.")

print(f"\n⚠️  Allgemeine Erkenntnisse:")
print("• Bei kleinen Datensätzen ist CV grundsätzlich weniger stabil")
print("• Ein einzelner Train-Test Split kann zufällig bessere Ergebnisse zeigen")
print("• CV gibt dennoch eine ehrlichere Einschätzung der Modell-Variabilität")
print("• Die optimale Fold-Anzahl muss experimentell bestimmt werden")

# Aktualisierte Variablen für späteren Vergleich (verwende die stabilste Methode)
if most_stable_idx == 0:
    cv_mean, cv_std = cv_mean_3, cv_std_3
elif most_stable_idx == 1:
    cv_mean, cv_std = cv_mean_5, cv_std_5
elif most_stable_idx == 2:
    cv_mean, cv_std = cv_mean_8, cv_std_8
else:
    cv_mean, cv_std = cv_mean_10, cv_std_10

print(f"\n📋 Für weitere Analysen verwenden wir {optimal_method} CV: {cv_mean:.4f} ± {cv_std:.4f}")

## 6. Evaluate Model Performance

Jetzt testen wir, wie gut unser neuronales Netz auf ungesehenen Daten funktioniert.

In [ ]:
# Vorhersagen auf Test- und Trainingsdaten
y_train_pred = mlp.predict(X_train_scaled)
y_test_pred = mlp.predict(X_test_scaled)

# Genauigkeiten berechnen
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("🎯 Model Performance:")
print(f"Training Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"Test Accuracy:     {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Overfitting Check
if train_accuracy - test_accuracy > 0.05:
    print("⚠️  Mögliches Overfitting (Training >> Test)")
elif abs(train_accuracy - test_accuracy) < 0.02:
    print("✅ Gute Generalisierung (Training ≈ Test)")
else:
    print("👍 Akzeptable Generalisierung")

# Vergleich mit Cross-Validation Ergebnissen
print(f"\n📊 Performance-Vergleich:")
print(f"Cross-Validation:  {cv_mean:.4f} ± {cv_std:.4f}")
print(f"Test Set:          {test_accuracy:.4f}")

cv_test_diff = abs(cv_mean - test_accuracy)
if cv_test_diff < cv_std:
    print("✅ Test Accuracy liegt im erwarteten CV-Bereich - gute Validierung!")
elif cv_test_diff < 2 * cv_std:
    print("👍 Test Accuracy nahe CV-Schätzung - akzeptable Abweichung")
else:
    print("⚠️  Test Accuracy weicht stark von CV ab - mögliche Datenprobleme")

print(f"Abweichung CV↔Test: {cv_test_diff:.4f} (Toleranz: ±{cv_std:.4f})")

# Detaillierter Classification Report
print("\n📊 Detaillierte Klassifikation (Test Set):")
print(classification_report(y_test, y_test_pred, target_names=iris.target_names))

In [ ]:
# Konfusionsmatrix visualisieren
plt.figure(figsize=(12, 5))

# Training Set Confusion Matrix
plt.subplot(1, 2, 1)
cm_train = confusion_matrix(y_train, y_train_pred)
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', 
           xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title(f'Training Set\nAccuracy: {train_accuracy:.3f}', fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Test Set Confusion Matrix
plt.subplot(1, 2, 2)
cm_test = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Greens',
           xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title(f'Test Set\nAccuracy: {test_accuracy:.3f}', fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.show()

# Fehleranalyse
test_errors = np.where(y_test != y_test_pred)[0]
if len(test_errors) > 0:
    print(f"\n🔍 Fehleranalyse: {len(test_errors)} falsche Vorhersagen im Test Set")
    for i in test_errors:
        actual = iris.target_names[y_test[i]]
        predicted = iris.target_names[y_test_pred[i]]
        print(f"Sample {i}: Actual={actual}, Predicted={predicted}")
else:
    print("\n🎉 Perfekte Klassifikation! Keine Fehler im Test Set.")

## 7. Make Predictions on New Data

Lass uns das trainierte Modell verwenden, um Vorhersagen für neue Iris-Blüten zu machen!

In [ ]:
# Beispiel-Blüten für Vorhersagen definieren
new_flowers = np.array([
    [5.1, 3.5, 1.4, 0.2],  # Typische Setosa
    [6.9, 3.1, 4.9, 1.5],  # Typische Versicolor  
    [6.3, 3.3, 6.0, 2.5],  # Typische Virginica
    [5.8, 2.7, 4.1, 1.0]   # Grenzfall
])

# Wichtig: Neue Daten mit dem gleichen Scaler transformieren!
new_flowers_scaled = scaler.transform(new_flowers)

# Vorhersagen machen
predictions = mlp.predict(new_flowers_scaled)
probabilities = mlp.predict_proba(new_flowers_scaled)

print("🔮 Vorhersagen für neue Iris-Blüten:")
print("=" * 60)

feature_names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']

for i, (flower, pred, probs) in enumerate(zip(new_flowers, predictions, probabilities)):
    print(f"\n🌸 Blüte {i+1}:")
    print(f"Features: {dict(zip(feature_names, flower))}")
    print(f"Vorhersage: {iris.target_names[pred]} (Klasse {pred})")
    print("Wahrscheinlichkeiten:")
    for j, (species, prob) in enumerate(zip(iris.target_names, probs)):
        print(f"  {species}: {prob:.3f} ({prob*100:.1f}%)")
    
    # Confidence-Level
    max_prob = np.max(probs)
    if max_prob > 0.9:
        confidence = "Sehr sicher 🎯"
    elif max_prob > 0.7:
        confidence = "Sicher 👍"
    elif max_prob > 0.5:
        confidence = "Unsicher 🤔"
    else:
        confidence = "Sehr unsicher ❓"
    
    print(f"Confidence: {confidence} (Max: {max_prob:.3f})")

In [ ]:
# Wahrscheinlichkeiten visualisieren
plt.figure(figsize=(12, 8))

# Subplot für jede neue Blüte
for i in range(len(new_flowers)):
    plt.subplot(2, 2, i+1)
    
    # Balkendiagramm der Wahrscheinlichkeiten
    bars = plt.bar(iris.target_names, probabilities[i], 
                  color=['red', 'green', 'blue'], alpha=0.7)
    
    # Vorhersage hervorheben
    predicted_idx = predictions[i]
    bars[predicted_idx].set_alpha(1.0)
    bars[predicted_idx].set_edgecolor('black')
    bars[predicted_idx].set_linewidth(2)
    
    plt.title(f'Blüte {i+1}: {iris.target_names[predicted_idx]}', fontweight='bold')
    plt.ylabel('Wahrscheinlichkeit')
    plt.ylim(0, 1)
    
    # Werte auf Balken anzeigen
    for bar, prob in zip(bars, probabilities[i]):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{prob:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Model-Insights anzeigen
print("\n🧠 Model-Insights:")
print(f"📊 Anzahl Parameter: {sum(w.size for w in mlp.coefs_) + sum(b.size for b in mlp.intercepts_)}")
print(f"🏗️  Schichtgrößen: Input({mlp.coefs_[0].shape[0]}) → Hidden({mlp.coefs_[0].shape[1]}) → Hidden({mlp.coefs_[1].shape[1]}) → Output({mlp.coefs_[2].shape[1]})")
print(f"⚙️  Aktivierungsfunktion: {mlp.activation}")
print(f"🎯 Training-Iterationen: {mlp.n_iter_}")
print(f"📈 Final Training Loss: {mlp.loss_curve_[-1]:.6f}")

## 🎯 Zusammenfassung und Erkenntnisse

**Was haben wir gelernt?**

### 📊 **Ergebnisse und deren Interpretation:**
- **Test-Genauigkeit:** Erscheint sehr hoch (>95%), aber Vorsicht bei der Interpretation!
- **Cross-Validation zeigt:** Die Performance kann stark variieren je nach Datenaufteilung
- **Wichtige Erkenntnis:** Ein einzelner Train-Test Split kann irreführend sein

### 🔍 **Cross-Validation Erkenntnisse:**
- **Variabilität der Ergebnisse:** CV zeigt Standardabweichungen zwischen verschiedenen Folds
- **Kleine Datensätze problematisch:** Sowohl wenige als auch viele Folds bringen Herausforderungen
- **Test-Accuracy könnte Zufall sein:** Die hohe Test-Performance ist möglicherweise ein "Glückstreffer"
- **CV gibt ehrlichere Einschätzung:** Zeigt die tatsächliche Variabilität der Modell-Performance

### 🧠 **Wichtige methodische Erkenntnisse:**

1. **Feature-Skalierung ist essentiell** für neuronale Netze
2. **Cross-Validation bei kleinen Datensätzen:** Muss experimentell optimiert werden
3. **Ein Test-Split reicht nicht:** Kann zufällig besser oder schlechter ausfallen
4. **Iris ist trügerisch einfach:** Kleine Datensätze verstärken Zufallseffekte

### 🔧 **Praktische Lektionen:**

- **Immer CV verwenden:** Besonders bei kleinen Datensätzen zur Validierung
- **Test-Ergebnisse hinterfragen:** Eine hohe Accuracy kann Zufall sein
- **Variabilität dokumentieren:** Standardabweichungen sind genauso wichtig wie Mittelwerte
- **Fold-Anzahl experimentell bestimmen:** Je nach Datensatz-Größe unterschiedlich optimal
- **Stratifiziert splitten:** Um Klassenverteilung zu erhalten

### ⚠️ **Kritische Reflexion:**
- **"Perfekte" Ergebnisse hinterfragen:** Bei kleinen Datensätzen oft nicht repräsentativ
- **CV als Reality-Check:** Zeigt die wahre Modell-Stabilität
- **Datensatz-Größe beachten:** Iris mit 150 Samples ist grenzwertig für robuste Schlüsse

### 🚀 **Nächste Schritte:**
- Teste das Modell mit **verschiedenen random_state Werten**
- Verwende **größere Datensätze** für stabilere Ergebnisse
- Experimentiere mit **verschiedenen CV-Strategien**
- **Dokumentiere immer die Variabilität**, nicht nur den besten Wert

**Neuronale Netze funktionieren auch bei kleinen Datensätzen - aber die Evaluierung erfordert besondere Sorgfalt! 🌸**